### Import Libraries

In [120]:
# Import twitter dependencies
import tweepy
from config import *

In [121]:
# Import transformation dependencies
import pandas as pd
from datetime import datetime, date
import json
import re 

In [122]:
# For sentiment and subjectivity analysis
from textblob import TextBlob

# Tweepy Setup

In [123]:
# Tweepy Setup
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [214]:
# Tweepy search parameters
search_terms = ['covid+vaccine OR covid-19+vaccine OR pfizer OR pfizer-biontech OR pfizerbiontech OR moderna OR astrazeneca OR astra+zeneca -filter:retweets']
end_date = date(2021,3,11)
max_id ='1370121363994841092'
limit = 1200 # this is my 15 minute limit :'( breaks if this becomes 1000

# Save files
output_csv ='csvs/tw61.csv'
output_json ='json/raw61.json'

# Functions

Collect and save data

In [215]:
# Pulls data from api and returns 2 lists:
    # one which is parsed for selected variables
    # one with all the raw data

data = []
raw = []
tweets = tweepy.Cursor(api.search, location='canada', max_id = max_id, place='canada', lang ='en', q=search_terms, tweet_mode='extended').items(limit)
for t in tweets: 
    tweet ={
        'created' :t.created_at,
        'id_str' : t.id_str,
        'text' : t.full_text,
        'user_id' :t.user.id,
        'screen_name': t.user.screen_name,
        'location': t.user.location,
        'followers_count': t.user.followers_count,
        'user_favourites_count': t.user.favourites_count,
        'time_zone' :t.user.time_zone,
        'geo_enabled' :t.user.geo_enabled,
        'verified' :t.user.verified,
        'status_count': t.user.statuses_count,
        'geo':t.geo,
        'coords':t.coordinates,
        'retweet_count': t.retweet_count,
        'tweet_favourite_count' : t.favorite_count
    }
    # add sorted data to one list 
    data.append(tweet)
    # keep raw data in case I realize later that I've removed something important 
    raw.append(t)
print('done!')

done!


In [216]:
# Add parsed data to dataframe, convert to csv
tweets_df = pd.DataFrame(data)

# Save raw data to json 
with open(output_json, 'w') as file:
    for resp in raw:
        jso = json.dumps(resp._json,sort_keys=True, indent=3)
        file.write(jso)
tweets_df.tail(5)

,created,id_str,text,user_id,screen_name,location,followers_count,user_favourites_count,time_zone,geo_enabled,verified,status_count,geo,coords,retweet_count,tweet_favourite_count
1195,2021-03-11 21:10:48,1370120043879563266,We’re expecting to see Governor Steve Sisolak ...,3246318390,JeremyChenKTNV,"Las Vegas, NV",1564,187,None,True,True,4163,None,None,0,2
1196,2021-03-11 21:10:48,1370120043426643968,RT @NC_Governor: Gov. Cooper announced that th...,2284720441,Just_Me_NCGirl,USA,100,1371,None,False,False,47720,None,None,187,0
1197,2021-03-11 21:10:48,1370120042151604235,RT @sullydish: Amazing. The NYT acknowledges b...,818053030868357120,cathyjones173,,545,42336,None,False,False,112161,None,None,82,0
1198,2021-03-11 21:10:48,1370120042009006089,39-Year-Old Healthy Utah Mother Dies After Tak...,358421123,Lisa4OR1st,"Kansas City, Kansas",2555,35213,None,True,False,53758,None,None,0,0
1199,2021-03-11 21:10:48,1370120041560223746,RT @Anaphylaxiscoms: If you have any questions...,1968773814,LynneRegentAC,South East UK,2800,11998,None,True,False,14885,None,None,7,0


### Clean tweets

In [217]:
# not working when I put these into a function
tweets_df = tweets_df.drop_duplicates(subset=['id_str'])
tweets_df = tweets_df.dropna(subset=['id_str', 'screen_name'])

In [218]:
# Makes tweets more readable 
def CleanTweets(txt): 
    # remove @ username 
    # txt = re.sub(r'@[A-Za-z0-9)]+','', txt)

    # remove @ from username
    txt = txt.replace('@','')    
    
    # remove RT    
    txt = re.sub(r'RT[\s]+','',txt)
    
    # remove # but leave txt
    txt = re.sub(r'#','', txt)
    
    # remove hyperlinks
    txt = re.sub(r'https?:\/\/\S+', '', txt)
    return txt

In [219]:
# Clean the text field 
tweets_df['text'] = tweets_df['text'].apply(CleanTweets)

In [220]:
# to get subjectivity 
def RateSubjectivity(txt):
    return TextBlob(txt).sentiment.subjectivity

In [221]:
# to get polarity
def RatePolarity(txt):
    return TextBlob(txt).sentiment.polarity

In [222]:
# add worded sentiment based on polarity score
def GetSentiment(num):
    if num < 0: 
        return 'negative' 
    elif num == 0:
        return 'neutral'
    else:
        return 'positive'

In [223]:
# Create Columns
def RateTweets(df):
    df['Subjectivity'] = df['text'].apply(RateSubjectivity)
    df['Polarity'] = df['text'].apply(RatePolarity)
    df['Sentiment'] = df['Polarity'].apply(GetSentiment)
    return df 

In [224]:
tweets_df = RateTweets(tweets_df)
tweets_df.to_csv(output_csv, index=False, encoding='UTF-8')
tweets_df.tail(5)

,created,id_str,text,user_id,screen_name,location,followers_count,user_favourites_count,time_zone,geo_enabled,verified,status_count,geo,coords,retweet_count,tweet_favourite_count,Subjectivity,Polarity,Sentiment
1195,2021-03-11 21:10:48,1370120043879563266,We’re expecting to see Governor Steve Sisolak ...,3246318390,JeremyChenKTNV,"Las Vegas, NV",1564,187,None,True,True,4163,None,None,0,2,0.000000,0.000000,neutral
1196,2021-03-11 21:10:48,1370120043426643968,NC_Governor: Gov. Cooper announced that thanks...,2284720441,Just_Me_NCGirl,USA,100,1371,None,False,False,47720,None,None,187,0,0.370833,-0.045833,negative
1197,2021-03-11 21:10:48,1370120042151604235,sullydish: Amazing. The NYT acknowledges biolo...,818053030868357120,cathyjones173,,545,42336,None,False,False,112161,None,None,82,0,0.950000,0.425000,positive
1198,2021-03-11 21:10:48,1370120042009006089,39-Year-Old Healthy Utah Mother Dies After Tak...,358421123,Lisa4OR1st,"Kansas City, Kansas",2555,35213,None,True,False,53758,None,None,0,0,0.250000,0.250000,positive
1199,2021-03-11 21:10:48,1370120041560223746,Anaphylaxiscoms: If you have any questions abo...,1968773814,LynneRegentAC,South East UK,2800,11998,None,True,False,14885,None,None,7,0,0.700000,0.500000,positive


In [213]:
tweets_df[['created','id_str','text','screen_name','location','Subjectivity','Polarity','Sentiment']].tail(5)

,created,id_str,text,screen_name,location,Subjectivity,Polarity,Sentiment
1195,2021-03-11 21:16:04,1370121367899795467,ddiamond: Trump officials last year conceived ...,MikeGeorgeCBS,New York City,0.083333,-0.050000,negative
1196,2021-03-11 21:16:04,1370121367010562060,bopanc: Serbia to start manufacturing Sinophar...,LostinEU,Brüssel,0.000000,0.000000,neutral
1197,2021-03-11 21:16:04,1370121366415020032,NoContextTFs: How it feels getting the COVID v...,alsoMike,Sweden,0.000000,0.000000,neutral
1198,2021-03-11 21:16:03,1370121366142382082,The retailer is in talks with the CDC about de...,dailyyonder,Rural America,0.500000,0.136364,positive
1199,2021-03-11 21:16:03,1370121363994841092,Nigeria Did Not Receive Batch Of AstraZeneca V...,MojiDelanoBlog,"Lagos,NIGERIA",0.000000,0.000000,neutral


# SQLAlchemy
### Connecting to a certain other host 

In [5]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [12]:
engine = create_engine(conn, echo=False)

In [13]:
Base = automap_base()
Base.prepare(engine, reflect=True)  

In [14]:
# Test connection
# it works!!!
# not anymore
# kill kernal if conn is wrong

Base.classes.keys()

['tw2']

### Don't run yet
Clean up your tweets! 

In [9]:
_df = pd.read_csv('csvs/tw2.csv')
_df.to_sql(name='tw2', con=engine, if_exists='append', index=True)
print('added!')

added!


In [ ]:
# session things
session = Session(engine)
session.close()
